# VCFX Benchmark Results Analysis

This notebook analyzes benchmark results comparing VCFX tools against standard bioinformatics tools like bcftools and vcftools.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# Load results
results_file = Path('../results.csv')
if results_file.exists():
    df = pd.read_csv(results_file)
    print(f"Loaded {len(df)} benchmark results")
    print(df.head())
else:
    print("No results file found. Please run the benchmarks first.")
    df = pd.DataFrame()

In [ ]:
# Filter completed tasks only
if not df.empty:
    completed_df = df[df['status'] == 'completed'].copy()
    print(f"Completed tasks: {len(completed_df)}")
    print(f"Skipped tasks: {len(df[df['status'] == 'skipped'])}")
    print(f"Failed tasks: {len(df[df['status'] == 'failed'])}")
    
    # Show tool distribution
    print("\nTool distribution:")
    print(completed_df['tool'].value_counts())


In [ ]:
# Performance comparison by tool
if not df.empty and 'completed_df' in locals():
    plt.figure(figsize=(12, 8))
    
    # Create bar plot
    if not completed_df.empty:
        # Group by task and show comparison
        plt.subplot(2, 1, 1)
        task_times = completed_df.groupby('task')['seconds'].first().sort_values()
        colors = ['#1f77b4' if 'vcfx' in task else '#ff7f0e' for task in task_times.index]
        bars = plt.bar(range(len(task_times)), task_times.values, color=colors)
        
        plt.title('Benchmark Results by Task')
        plt.ylabel('Time (seconds)')
        plt.xticks(range(len(task_times)), task_times.index, rotation=45, ha='right')
        
        # Add value labels on bars
        for bar, time in zip(bars, task_times.values):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    f'{time:.2f}s', ha='center', va='bottom')
        
        # Tool comparison
        plt.subplot(2, 1, 2)
        tool_avg = completed_df.groupby('tool')['seconds'].mean().sort_values()
        colors = ['#1f77b4' if 'vcfx' in tool else '#ff7f0e' for tool in tool_avg.index]
        bars = plt.bar(range(len(tool_avg)), tool_avg.values, color=colors)
        
        plt.title('Average Performance by Tool')
        plt.ylabel('Average Time (seconds)')
        plt.xticks(range(len(tool_avg)), tool_avg.index)
        
        # Add value labels on bars
        for bar, time in zip(bars, tool_avg.values):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    f'{time:.2f}s', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
else:
    print("No data to plot")
